In [10]:
import os
import requests
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from zoneinfo import ZoneInfo
from dotenv import load_dotenv
from io import BytesIO

from read_write_google_sheet import read_google_sheet

# --- Load environment variables ---
load_dotenv()
TELEGRAM_BOT_TOKEN = os.getenv("telegram_bot_token")
TELEGRAM_CHAT_ID = os.getenv("channel_id")  # e.g., '@yourchannel'

# --- Time info ---
NPT = ZoneInfo("Asia/Kathmandu")
now_npt = datetime.now(NPT)
current_time_str = now_npt.strftime("%H:%M")
current_date = now_npt.strftime("%Y-%m-%d")

# --- Send image to Telegram ---
def send_plot_to_telegram(title, fig):
    telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendPhoto"
    
    img_bytes = fig.to_image(format="png")
    buf = BytesIO(img_bytes)
    buf.seek(0)

    files = {'photo': buf}
    data = {
        "chat_id": TELEGRAM_CHAT_ID,
        "caption": f"{title} ({current_date} {current_time_str})",  # Removed HTML parse mode
    }

    response = requests.post(telegram_url, data=data, files=files)
    if response.ok:
        print(f"Sent: {title}")
    else:
        print(f"[ERROR] {title}: {response.text}")

# And when creating the oversold plot:
oversold = latest_rsi[latest_rsi < 30].sort_values().dropna()
if not oversold.empty:
    oversold_fig = go.Figure()
    oversold_fig.add_trace(go.Bar(
        x=oversold.index,
        y=oversold.values,
        marker_color='blue',
        text=oversold.values.round(2),
        textposition='auto'
    ))
    
    oversold_fig.update_layout(
        title="Oversold Stocks (RSI < 30)",
        yaxis_title="RSI",
        xaxis_title="Ticker",
        template="plotly_white",
        height=600
    )
    
    send_plot_to_telegram("📉 Oversold Stocks (RSI < 30)", oversold_fig)
else:
    print("No oversold stocks found")

# --- Create Plotly Bar Chart ---
def create_bar_chart(data, title, ylabel, color='skyblue'):
    if data.empty:
        return None
        
    data = data.head(10)  # Ensure only top 10
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=data.index,
        y=data.values.round(2),
        marker_color=color,
        text=data.values.round(2),
        textposition='auto'
    ))
    
    fig.update_layout(
        title=title,
        yaxis_title=ylabel,
        xaxis_title="Ticker",
        template="plotly_white",
        height=600
    )
    
    return fig

# --- Create Table ---
def create_table(data, title, columns):
    if data.empty:
        return None
        
    data = data.head(10)  # Ensure only top 10
    fig = go.Figure(data=[go.Table(
        header=dict(values=columns),
        cells=dict(values=[data.index, data.values.round(2)])
    )])
    
    fig.update_layout(
        title=title,
        height=400
    )
    
    return fig

# --- Load data ---
sheet_id = "1n_QX2H3HEM1wYbEQmHV4fYBwfDzd19sBEiOv4MBXrFo"
df = read_google_sheet(sheet_id)
df['Date'] = pd.to_datetime(df['Date'])
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df['Turnover'] = pd.to_numeric(df['Turnover'], errors='coerce')

pivot = df.pivot_table(values='Close', index='Date', columns='Ticker')
daily_pct = pivot.pct_change(fill_method= None)
latest_change = daily_pct.iloc[-1] * 100

# --- Top Gainers & Losers ---
gainers = latest_change.sort_values(ascending=False).dropna()
losers = latest_change.sort_values().dropna()

gainers_fig = create_bar_chart(gainers, "Top 10 Gainers (%)", "Daily % Change", color='green')
if gainers_fig:
    send_plot_to_telegram("📈 Top 10 Gainers (%)", gainers_fig)

losers_fig = create_bar_chart(losers, "Top 10 Losers (%)", "Daily % Change", color='red')
if losers_fig:
    
    send_plot_to_telegram("📉 Top 10 Losers (%)", losers_fig)

# --- RSI ---
rsi_df = pivot.apply(lambda x: ta.rsi(x, length=14))
latest_rsi = rsi_df.iloc[-1]
overbought = latest_rsi[latest_rsi > 70].sort_values(ascending=False).dropna()
oversold = latest_rsi[latest_rsi < 30].sort_values().dropna()

Sent: 📉 Oversold Stocks (RSI < 30)
Sent: 📈 Top 10 Gainers (%)
Sent: 📉 Top 10 Losers (%)


In [11]:
oversold_fig = create_bar_chart(oversold, "Oversold Stocks (RSI < 30)", "RSI", color='blue')
if oversold_fig:
    send_plot_to_telegram("📉 Oversold Stocks (RSI < 30)", oversold_fig)


Sent: 📉 Oversold Stocks (RSI < 30)


In [9]:
oversold

Ticker
TPC       19.793365
NYADI     20.442559
CHL       21.831823
RHGCL     22.251204
JBBD87    25.530147
UNL       27.793860
SBID89    27.986941
BHDC      28.419842
RFPL      28.770223
MKHC      29.845576
SIKLES    29.980379
Name: 2025-05-27 00:00:00, dtype: float64